In [1]:
!pip list

Package             Version
------------------- ---------
altair              4.1.0
appnope             0.1.2
argon2-cffi         20.1.0
astor               0.8.1
async-generator     1.10
attrs               20.3.0
backcall            0.2.0
base58              2.1.0
bleach              3.3.0
blinker             1.4
blis                0.7.4
cachetools          4.2.1
catalogue           2.0.1
certifi             2020.12.5
cffi                1.14.4
chardet             4.0.0
click               7.1.2
cymem               2.0.5
decorator           4.4.2
defusedxml          0.6.0
docopt              0.6.2
en-core-web-sm      3.0.0
entrypoints         0.3
gitdb               4.0.5
GitPython           3.1.12
idna                2.10
ipykernel           5.4.3
ipython             7.20.0
ipython-genutils    0.2.0
ipywidgets          7.6.3
jedi                0.18.0
Jinja2              2.11.3
joblib              1.0.0
jsonschema          3.2.0
jupyter             1.0.0
jupyter-client      6.1.11


In [2]:
import spacy
from spacy import displacy
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
nlp = spacy.load('en_core_web_sm')

In [4]:
text = "Apple is a great company. However, I am unaware of it."

In [5]:
doc = nlp(text)

In [6]:
for token in doc:
    print(token.lemma_)

Apple
be
a
great
company
.
however
,
I
be
unaware
of
it
.


In [7]:
displacy.render(doc, style='ent')

In [8]:
data_yelp = pd.read_csv('database/yelp.txt', sep='\t', header = None)
columns_name = ['Review', 'Sentiment']
data_yelp.columns = columns_name
data_yelp.head()


,Review,Sentiment
0,Wow... Loved this place.,1.0
1,I learned that if an electric slicer is used t...,NaN
2,But they don't clean the chiles?,NaN
3,Crust is not good.,0.0
4,Not tasty and the texture was just nasty.,0.0


In [9]:
data_amazon = pd.read_csv('database/amazon.txt', sep='\t', header = None, error_bad_lines=False)
columns_name = ['Review', 'Sentiment']
data_amazon.columns = columns_name

In [10]:
data_amazon.head()

,Review,Sentiment
0,So there is no way for me to plug it in here i...,0.0
1,"Good case, Excellent value.",1.0
2,Great for the jawbone.,1.0
3,When I got this item it was larger than I thou...,NaN
4,(I looked for one that specifically said DCU-6...,NaN


In [11]:
data_imdb = pd.read_csv('database/imdb.txt', sep='\t', header = None, error_bad_lines=False)
columns_name = ['Review', 'Sentiment']
data_imdb.columns = columns_name
data_imdb.head()

,Review,Sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [12]:
data_yelp.shape, data_amazon.shape, data_imdb.shape


((3729, 2), (14606, 2), (748, 2))

In [13]:
import string
punct = string.punctuation
punct

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [14]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
print(stopwords)

['seemed', 'first', 'mine', "'ll", 'whence', 'what', 'per', 'whatever', 'not', '’d', 'to', 'someone', 'these', 'then', 'will', 'nobody', 'rather', 'but', 'so', "'d", 'except', 'somehow', 'amount', 'nowhere', 'out', 'too', 'everything', 'even', 'nine', 'if', 'alone', 'thus', 'of', 'else', 'back', 'be', 'into', 'say', 'herein', 'however', 'serious', 'few', 'who', 'also', 'six', '‘d', 'hence', 'she', 'at', '’m', 'doing', 'under', 'either', 'do', 'three', 'much', 'us', 'beside', 'both', 'all', 'thereupon', 'why', 'must', 'thru', 'off', 'whereafter', 'see', 'therefore', 'whereupon', '’re', 'and', 'nevertheless', 'whither', 'below', 'please', 'really', 'it', 'mostly', 'noone', 'each', 'within', 'cannot', 'former', 'almost', 're', 'unless', 'after', 'have', 'made', 'empty', 'with', 'top', 'during', '‘m', 'though', '’ve', 'everywhere', 'down', 'anyone', 'part', 'i', 'whereas', 'sixty', 'otherwise', 'them', 'eight', 'somewhere', 'move', 'give', 'take', 'does', 'once', 'in', 'hereupon', 'seeming

class customNlp:
    
    def __init__(self):
        pass
    
    def text_data_cleaning(self,sentence):
        doc = nlp(sentence)

        tokens = []
        for token in doc:
            if token.lemma_ != "-PRON-":
                temp = token.lemma_.lower().strip()
            else:
                temp = token.lower_
            tokens.append(temp)

        cleaned_tokens = []
        for token in tokens:
            if token not in stopwords and token not in punct:
                cleaned_tokens.append(token)
        return cleaned_tokens

#myclass = customNlp()
mysentence = customNlp().text_data_cleaning("Hello how are you. I love my girl")
print(mysentence)

In [15]:
from build_library.utils import customNlp

In [16]:
mysentence = customNlp().text_data_cleaning("Hello how are you. I love my girl")
print(mysentence)

['hello', 'love', 'girl']


In [17]:
data = data_yelp.append([data_amazon, data_imdb], ignore_index=True)
data.shape

(19083, 2)

In [18]:
data['Sentiment'].value_counts()


1.0    1386
0.0    1362
Name: Sentiment, dtype: int64

In [19]:
data.isnull().sum()

Review           0
Sentiment    16335
dtype: int64

In [20]:
data['Sentiment'] 

0        1.0
1        NaN
2        NaN
3        0.0
4        0.0
        ... 
19078    0.0
19079    0.0
19080    0.0
19081    0.0
19082    0.0
Name: Sentiment, Length: 19083, dtype: float64

In [21]:
dataready = data[data['Sentiment'].notnull()]

In [22]:
dataready.isnull().sum()

Review       0
Sentiment    0
dtype: int64

In [23]:
tfidf = TfidfVectorizer(tokenizer = customNlp().text_data_cleaning)
classifier = LinearSVC(penalty='l2',C=1)

In [24]:
X = dataready['Review']
y = dataready['Sentiment']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)
X_train.shape, X_test.shape

((2198,), (550,))

In [26]:
clf = Pipeline([('tfidf', tfidf), ('clf', classifier)])
clf.fit(X_train, y_train)

Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<bound method customNlp.text_data_cleaning of <build_library.utils.customNlp object at 0x13347af40>>)),
                ('clf', LinearSVC(C=1))])

In [27]:
y_pred = clf.predict(X_test)

In [28]:
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

         0.0       0.78      0.81      0.80       285
         1.0       0.79      0.75      0.77       265

    accuracy                           0.79       550
   macro avg       0.79      0.78      0.78       550
weighted avg       0.79      0.79      0.79       550



In [29]:
confusion_matrix(y_test, y_pred)


array([[232,  53],
       [ 65, 200]])

In [30]:
clf.predict(['Wow, this is amazing lesson'])


array([1.])

In [31]:
clf.predict(['Wow, this sucks'])


array([0.])

In [32]:
clf.predict(['Loved it. Amazing'])


array([1.])

In [33]:
import pickle

In [34]:
pickle.dump([customNlp,clf], open("pipeline.pickle", "wb"))

In [35]:
file = open('pipeline.pickle', 'rb')

# dump information to that file
mytest = pickle.load(file)

# close the file
file.close()

In [36]:
print(mytest)

[<class 'build_library.utils.customNlp'>, Pipeline(steps=[('tfidf',
                 TfidfVectorizer(tokenizer=<bound method customNlp.text_data_cleaning of <build_library.utils.customNlp object at 0x134916a90>>)),
                ('clf', LinearSVC(C=1))])]


In [37]:
mytest[0]().text_data_cleaning("Hello how are you. I love my girl")

['hello', 'love', 'girl']